# 类的方法

## 方法（Method）
---

In [1]:
class MyClass(object):
    i = 1
    def foo(self):
        pass
mc = MyClass()
mc.foo()

上面定义了一个类`MyClass`，类中定义了一个方法`foo`,如果像函数一样对`foo`自身的调用都会失败：

In [2]:
foo()

NameError: name 'foo' is not defined

直接调用引发了NameError的异常，因为在全局函数中没有这样的函数存在。也就是说`foo`是一个方法表示它属于一个类而不是全局空间中的名字。

In [3]:
MyClass.foo()

TypeError: foo() missing 1 required positional argument: 'self'

In [4]:
MyClass.i

1

直接调用`MyClass.foo()`会抛出TypeError的异常，这是因为为了与OOP惯例保持一致，Python严格要求：没有实例的方法是不能被调用的。这种限制即Python所描述的绑定概念（binding），在此，**方法必须绑定（到一个实例）才能直接被调用**。

## 绑定和方法调用
---

+ 方法仅仅是类内部定义的函数（这意味着方法是类属性而不是实例属性）。
+ 方法只有在其所属的类拥有实例时才能够被调用，当存在一个实例时，方法才被认为是绑定到那个实例了，否则就是未绑定的。
+ 任何一个方法定义中的第一个参数都是`self`，他表示调用此方法的实例对象

### 调用绑定方法

首先看一个例子<sup>[1]</sup>:

In [5]:
class People:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def talk(self):
        pass

A = People('xiaoming',18)
print(A.talk)    

<bound method People.talk of <__main__.People object at 0x0000017F5004C358>>


从上面的输出结果来看，`talk()`这个类中的方法，是绑定给对象使用的。下面，再看看另外一种情况：

In [6]:
class People:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def talk():
        pass
    
B = People('xiaohong',16)
print(B.talk)
print(People.talk)

<bound method People.talk of <__main__.People object at 0x0000017F5004CE80>>
<function People.talk at 0x0000017F5004F400>


　现在，我们将`talk()`函数的`self`变量去掉，使其为类中的函数，`B.talk`仍然是bound method。这说明，不管是类中的方法，还是类中函数，默认情况下都是绑定给对象使用的。绑定给对象使用有一种好处，那就是不用手动将对象传入。对象是自动传到类中。而如果类来调用类中的方法`People.talk`，那么这个方法仅仅只是一个函数，那么既然是函数，就不会有自动传值这一功能。

#### Reference

1. https://www.cnblogs.com/vipchenwei/p/7126772.html
2. https://docs.python.org/3/library/functions.html?highlight=classmethod#staticmethod
3. https://docs.python.org/3/library/functions.html?highlight=classmethod#classmethod
4. http://www.wklken.me/posts/2013/12/22/difference-between-staticmethod-and-classmethod-in-python.html